In [2]:
import torch
from torch import nn
from d2l import torch as d2l

def dropout_layer(X, dropout):
    assert 0 <= dropout <= 1
    if dropout == 1:
        return torch.zeros_like(X)
    if dropout == 0:
        return X
    mask = (torch.Tensor(X.shape).uniform_(0, 1) > dropout).float()
    return mask * X / (1.0 - dropout)

In [3]:
X = torch.arange(16, dtype=torch.float32).reshape((2, 8))
print(X)
print(dropout_layer(X, 0.))
print(dropout_layer(X, 0.5))
print(dropout_layer(X, 1.))

tensor([[ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.],
        [ 8.,  9., 10., 11., 12., 13., 14., 15.]])
tensor([[ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.],
        [ 8.,  9., 10., 11., 12., 13., 14., 15.]])
tensor([[ 0.,  2.,  0.,  0.,  0., 10., 12., 14.],
        [ 0.,  0.,  0., 22., 24., 26., 28.,  0.]])
tensor([[0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.]])


In [5]:
num_inputs, num_outputs, num_hiddens1, num_hiddens2 = 784, 10, 256, 256

dropout1, dropout2 = 0.2, 0.5

class Net(nn.Module):
    def __init__(self, num_inputs, num_outputs, num_hiddens1, num_hiddens2,
                 is_training=True):
        super(Net, self).__init__()
        self.num_inputs = num_inputs
        self.training = is_training
        self.lin1 = nn.Linear(num_inputs, num_hiddens1)
        self.lin2 = nn.Linear(num_hiddens1, num_hiddens2)
        self.lin3 = nn.Linear(num_hiddens2, num_outputs)
        self.relu = nn.ReLU()

    def forward(self, X):
        H1 = self.relu(self.lin1(X.reshape((-1, self.num_inputs))))
        if self.training == True:
            H1 = dropout_layer(H1, dropout1)
        H2 = self.relu(self.lin2(H1))
        if self.training == True:
            H2 = dropout_layer(H2, dropout2)
        out = self.lin3(H2)
        return out

net = Net(num_inputs, num_outputs, num_hiddens1, num_hiddens2)

In [ ]:
num_epochs, lr, batch_size = 10, 0.5, 256
loss = nn.CrossEntropyLoss()
train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size)
trainer = torch.optim.SGD(net.parameters(), lr=lr)
d2l.train_ch3(net, train_iter, test_iter, loss, num_epochs, trainer)

## numpy version

In [34]:
import numpy as np
def dropout_layer_numpy(X, dropout):
    assert 0 <= dropout <= 1
    mask = (np.random.uniform(size = X.shape, low= 0, high =1) > dropout).astype(float)
    return mask * X / (1.0 - dropout)

In [35]:
X = np.array([[2,3,4],[-2,1,3]])
dropout_layer_numpy(X, 0.5)

array([[ 0.,  6.,  8.],
       [-0.,  0.,  6.]])

## softmax

In [48]:
import torch
def softmax(x):
    maxes = torch.max(x, 1, keepdim=True)[0]
#     print(torch.max(x, 1, keepdim=True))
    print(maxes, 'maxes')
    x_exp = torch.exp(x-maxes)
    x_exp_sum = torch.sum(x_exp, 1, keepdim=True)

    return x_exp/x_exp_sum
x = torch.Tensor([[2,3,4],[-2,1,3]])
softmax(x)

tensor([[4.],
        [3.]]) maxes


tensor([[0.0900, 0.2447, 0.6652],
        [0.0059, 0.1185, 0.8756]])

In [46]:
def softmax_numpy(x):
    
    maxes = np.max(x, 1, keepdims=True)
#     print(maxes, 'maxes')    
    x_exp = np.exp(x-maxes)
    x_exp_sum = np.sum(x_exp, 1, keepdims=True)

    return x_exp/x_exp_sum
x = np.array([[2,3,4],[-2,1,3]])
softmax_numpy(x)

[4] maxes


array([[0.09003057, 0.24472847, 0.66524096],
       [0.00589975, 0.11849965, 0.8756006 ]])

## sigmoid

In [52]:
def sigmoid(x):
    return 1/(1 + torch.exp(-x))
x = torch.Tensor([[2,3,4],[-2,1,3]])
print(sigmoid(x))

tensor([[0.8808, 0.9526, 0.9820],
        [0.1192, 0.7311, 0.9526]])


In [51]:
def sigmoid_numpy(x):
    return 1/(1 + np.exp(-x))
x = np.array([[2,3,4],[-2,1,3]])
sigmoid_numpy(x)

array([[0.88079708, 0.95257413, 0.98201379],
       [0.11920292, 0.73105858, 0.95257413]])